In [1]:
# Download voxconverse dataset
!wget --load-cookies /tmp/cookies.txt --no-verbose "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate --no-verbose 'https://docs.google.com/uc?export=download&id=1jkmsypHYrljIlDuuCfe2vABez1Own5r9' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1jkmsypHYrljIlDuuCfe2vABez1Own5r9" -O voxconverse_dev_wav.zip && rm -rf /tmp/cookies.txt

# Unzip data files
!unzip -o -q voxconverse_dev_wav.zip -d ./

# Remove zip file
!rm voxconverse_dev_wav.zip

# Pull labels from github
!git clone https://github.com/joonson/voxconverse.git

!pip install torchaudio -q --no-deps
!pip install speechbrain -q
!pip install spectralcluster -q
!pip install pyannote.metrics -q

2021-04-03 02:05:04 URL:https://doc-0g-34-docs.googleusercontent.com/docs/securesc/uaghsg51fepid8lku9bgv4dvj27m10i3/hkeg0ds21fm265tsbnfv9vg1gq36iu1t/1617415425000/03361959783973937060/08834522133087532980Z/1jkmsypHYrljIlDuuCfe2vABez1Own5r9?e=download&nonce=o07rfs90c3ecs&user=08834522133087532980Z&hash=6i0o6sksrn3fpcr9c392os5i8ghjl0tn [1988647478] -> "voxconverse_dev_wav.zip" [1]
Cloning into 'voxconverse'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 224 (delta 140), reused 224 (delta 140), pack-reused 0
Receiving objects: 100% (224/224), 97.46 KiB | 98.00 KiB/s, done.
Resolving deltas: 100% (140/140), done.
     |████████████████████████████████| 1.9MB 4.4MB/s 
     |████████████████████████████████| 348kB 5.7MB/s 
     |████████████████████████████████| 1.2MB 12.0MB/s 
     |████████████████████████████████| 645kB 24.3MB/s 
     |████████████████████████████████| 102kB 8.6MB/s 

In [2]:
from __future__ import print_function, division

import os
import torch
import torchaudio
import numpy as np
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

# Load VAD Model
# https://github.com/snakers4/silero-vad
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True)
(get_speech_ts, _, read_audio, _, _, _) = utils

Downloading: "https://github.com/snakers4/silero-vad/archive/master.zip" to /root/.cache/torch/hub/master.zip


In [4]:
!mkdir vad

In [ ]:
root_dir='./audio/'
filelist = sorted(os.listdir(root_dir))

for idx in tqdm(range(len(filelist))):
    audio_path = os.path.join(root_dir, filelist[idx])
    audio = read_audio(audio_path, target_sr=16000)
    speech_timestamps = get_speech_ts(audio, model, num_steps=4)
    np.save("vad/" + filelist[idx][:-3] + "npy", np.array(speech_timestamps))

In [6]:
!zip -r -q vad.zip vad